In [ ]:
!pip3 install transformers

In [70]:
import pandas as pd

# panda view settings
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

In [71]:
df = pd.read_csv("./data/toxicbias_train.csv")
df_bias = df[df['bias'] == 'bias']
df_neutral = df[df['bias'] == 'neutral']


In [72]:
# Create the five catgories to split the data into and store them in a dictionary
categories = ['religion', 'race', 'lgbtq', 'political', 'gender']

category_dfs = {category: pd.DataFrame(columns=df.columns) for category in ['religion', 'race', 'lgbtq', 'political', 'gender']}


In [78]:
# Split categories and add rows to the corresponding dataframes in dictionary
def split_categories_and_add_rows(row):
    categories = row['category'].split(',')
    for category in categories:
        category = category.strip()  # Remove leading/trailing spaces
        if category in category_dfs:
            category_dfs[category] = category_dfs[category].append(row, ignore_index=True)

df_bias.apply(split_categories_and_add_rows, axis=1)

# Print the shape of each category dataframe
for category, category_df in category_dfs.items():
    print(f"Dataframe for {category} category:")
    print(f"Shape: {category_df.shape}")
    print("\n")

Dataframe for religion category:
Shape: (7542, 6)


Dataframe for race category:
Shape: (10608, 6)


Dataframe for lgbtq category:
Shape: (2718, 6)


Dataframe for political category:
Shape: (1032, 6)


Dataframe for gender category:
Shape: (1758, 6)




In [79]:
category_dfs['race'].sample(5)

,comment_text,identity_attack_x,bias,target,category,rationale
10102,utilitas wishes to know why white people are privileged. Boggle.,0.500000,bias,whites,race,prejudice against whites
5477,Pretty clear that Netanyahu is a fascist wishing to cleanse Israel of anyone who is not a white EuroJew. If this was anywhere else but Israel the CIA and MI6 would be working on regime change just as they have in most of the rest of the Middle East.,0.500000,bias,"whites,jews","race,religion",prejudice against whites and jews
9165,Watch out for those white males. They are considered the enemy now a days.,0.800000,bias,whites,race,prejudice against whites
5331,I believe all Indians should go back to their reserves. there is no need for them to have an education Indians never went to school before the white man came here. If all Indians were back on the reserve they could immerse themselves in their culture and be happy. Canadians will continue to carry them on our backs.,0.500000,bias,"indians,whites",race,prejudice against whites and indians
6945,Black crime has gotten way out of hand as I stated previously. Evidently facts and statistics are uncivil. Pathetic,0.578947,bias,blacks,race,prejudice against blacks
